In [1]:
import tensorflow as tf
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
from collections import Counter

import pickle

2022-12-23 18:29:08.145059: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
images=[]
labels=[]

feature_dictionary = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'label_normal': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string)
    }


def _parse_function(example, feature_dictionary=feature_dictionary):
    parsed_example = tf.io.parse_example(example, feature_dictionary)
    return parsed_example

def read_data(filename):
    full_dataset = tf.data.TFRecordDataset(filename,num_parallel_reads=tf.data.experimental.AUTOTUNE)
    full_dataset = full_dataset.shuffle(buffer_size=31000)
    full_dataset = full_dataset.cache()
    print("Size of Training Dataset: ", len(list(full_dataset)))

    full_dataset = full_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    print(full_dataset)
    for image_features in full_dataset:
        image = image_features['image'].numpy()
        image = tf.io.decode_raw(image_features['image'], tf.uint8)
        image = tf.reshape(image, [299, 299])        
        image=image.numpy()
        image=cv2.resize(image,(100,100))
        image=cv2.merge([image,image,image])        
        #plt.imshow(image)
        images.append(image)
        labels.append(image_features['label_normal'].numpy())

filenames=['./data/training10_3/training10_3.tfrecords',
#          './data/training10_1/training10_1.tfrecords',
#           './data/training10_2/training10_2.tfrecords',
#           './data/training10_3/training10_3.tfrecords',
#           './data/training10_4/training10_4.tfrecords']
          ]

for file in filenames:
    read_data(file)
    
print(len(images))
print(len(labels))

2022-12-23 18:30:17.512771: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-23 18:30:17.536074: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-23 18:30:17.660187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 18:30:17.660874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 960M computeCapability: 5.0
coreClock: 1.176GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 74.65GiB/s
2022-12-23 18:30:17.660974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-23 18:30:21.111631: I tensorflow/stream_executor/platform/defa

2022-12-23 18:33:49.155369: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 8749 of 31000
2022-12-23 18:33:59.193309: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 9619 of 31000
2022-12-23 18:34:09.161830: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 10506 of 31000
2022-12-23 18:34:16.887378: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:230] Shuffle buffer filled.


Size of Training Dataset:  11177
<ParallelMapDataset shapes: {image: (), label: (), label_normal: ()}, types: {image: tf.string, label: tf.int64, label_normal: tf.int64}>
11177
11177


In [3]:
images = np.array(images)
labels = np.array(labels)

In [4]:
images.shape

(11177, 100, 100, 3)

In [5]:
images[:, 0].mean(), images[:, 0].std()

(89.34033193164534, 49.839855611442815)

In [6]:
pickle.dump((images, labels), open('images_10_3_normal.p', 'wb'))